**GAN**

In [17]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

In [44]:
seq_train = np.load("AMPLify-Feedback/processed_data/test/seq_cv.npy")
state_train = np.load("AMPLify-Feedback/processed_data/test/state_cv.npy")
label_train = np.load("AMPLify-Feedback/processed_data/test/label_cv.npy")
seq_val = np.load("AMPLify-Feedback/processed_data/test/seq_test.npy")
state_val = np.load("AMPLify-Feedback/processed_data/test/state_test.npy")
label_val = np.load("AMPLify-Feedback/processed_data/test/label_test.npy")
seq_train = tf.one_hot(seq_train.squeeze(), depth=43)
seq_val = tf.one_hot(seq_val.squeeze(), depth=43)

In [54]:
seq_train

<tf.Tensor: shape=(24569, 190, 43), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 

In [36]:
# Generator
def build_generator(seq_length, depth, latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(seq_length * depth, activation='tanh'))
    model.add(layers.Reshape((seq_length, depth)))
    return model

In [37]:
# Discriminator
def build_discriminator(seq_length, depth):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(seq_length, depth)))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [38]:
# GAN
def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    discriminator.trainable = False
    gan_input = layers.Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = tf.keras.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

In [39]:
def train_gan(generator, discriminator, gan, seq_train, epochs, batch_size, latent_dim):
    for epoch in range(epochs):
        for i in range(0, seq_train.shape[0], batch_size):
            real_sequences = seq_train[i:i + batch_size]
            current_batch_size = real_sequences.shape[0]

            # Generate Fake sequence
            noise = np.random.normal(0, 1, (current_batch_size, latent_dim))
            generated_sequences = generator.predict(noise)

            # Labels for real and fake data
            real_labels = np.ones((current_batch_size, 1))
            fake_labels = np.zeros((current_batch_size, 1))

            # Train discriminator
            d_loss_real = discriminator.train_on_batch(real_sequences, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_sequences, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (current_batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((current_batch_size, 1)))

            # Print the progress
            print(f"Epoch {epoch+1}/{epochs}, Batch {i//batch_size+1}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

In [40]:
seq_train.shape

TensorShape([24569, 190, 43])

In [49]:
latent_dim = 10  # Example latent dimension
seq_length = seq_train.shape[1]
depth = 43

generator = build_generator(seq_length, depth, latent_dim)
discriminator = build_discriminator(seq_length,depth)
gan = compile_gan(generator, discriminator)

# Train GAN
train_gan(generator, discriminator, gan, seq_train, epochs=30, batch_size=32, latent_dim=latent_dim)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 52ms/step
Epoch 27/30, Batch 573, Discriminator Loss: [0.00596723 1.        ], Generator Loss: 7.692093372344971
1/1 [==============================] - 0s 29ms/step
Epoch 27/30, Batch 574, Discriminator Loss: [0.01334999 1.        ], Generator Loss: 8.329419136047363
1/1 [==============================] - 0s 35ms/step
Epoch 27/30, Batch 575, Discriminator Loss: [8.7827479e-04 1.0000000e+00], Generator Loss: 8.235692977905273
1/1 [==============================] - 0s 49ms/step
Epoch 27/30, Batch 576, Discriminator Loss: [3.96350311e-04 1.00000000e+00], Generator Loss: 8.598709106445312
1/1 [==============================] - 0s 58ms/step
Epoch 27/30, Batch 577, Discriminator Loss: [8.22208531e-04 1.00000000e+00], Generator Loss: 8.234081268310547
1/1 [==============================] - 0s 49ms/step
Epoch 27/30, Batch 578, Discriminator Loss: [0.00162888 1.        ], Generator Loss: 7.4489068984985

In [50]:
# After GAN is trained

def generate_sequences(generator, latent_dim, num_sequences):
    noise = np.random.normal(0, 1, (num_sequences, latent_dim))
    generated_sequences = generator.predict(noise)
    return generated_sequences


In [51]:
generated_seqs = generate_sequences(generator, latent_dim, num_sequences=10)

1/1 [==============================] - 0s 21ms/step


In [52]:
generated_seqs

array([[[-0.13389806, -0.00868082,  0.05481961, ...,  0.0820306 ,
         -0.04451147, -0.08700304],
        [-0.00214781,  0.01384653,  0.11464857, ...,  0.07179324,
         -0.04101948,  0.01057216],
        [ 0.00933576, -0.06249411, -0.49590158, ..., -0.02723819,
         -0.00311639, -0.03146577],
        ...,
        [ 0.9926709 ,  0.0744976 ,  0.07956082, ...,  0.02003127,
          0.04012141, -0.00879075],
        [ 0.98905957, -0.0066167 , -0.01975166, ..., -0.00340328,
         -0.07084771,  0.02803356],
        [ 0.987495  , -0.12202229,  0.112914  , ..., -0.07878347,
         -0.0105056 ,  0.07059029]],

       [[-0.15630676, -0.00518582,  0.02611491, ...,  0.06730221,
         -0.05616658, -0.09483384],
        [-0.00438303,  0.01867869,  0.12643312, ...,  0.06229681,
         -0.03994345,  0.01243595],
        [ 0.01646501, -0.06215589, -0.52843046, ..., -0.02946489,
         -0.00718941, -0.03792197],
        ...,
        [ 0.9941855 ,  0.07079332,  0.08867102, ...,  

In [60]:
np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_10",generated_seqs)

In [61]:
#Convert to One_hot
generated_seqs_one_hot = tf.one_hot(generated_seqs.squeeze(), depth=43)
np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_one_hot_10",generated_seqs_one_hot)